In [4]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.optimize import minimize

## Problema principal

Buscamos minimizar las cantidades $$K_1=(4b+c-4a)C_1$$ y $$K_2=(4b+c-4a)C_2$$ provenientes de la función
$$K(\tau)=K_1+K_2\tau^{1/2}-\tfrac{2}{3\sqrt{3}}(\tau-1)^{3/2}.$$
Para esto, establecemos el problema de minimización $(P)$ dado por
$$\min_{(a, b, c, C_1, C_2)\in X}\ K_1^2+K_2^2$$
donde $X\subset \mathbb{R}^5$ es el espacio de restricciones del problema.

## Problema de optimización

A continuación se define la función _objetivo_ del problema de optimización, seguido de las _constraints_ de dicho problema.

*_Observación_*: por comodidad, en el código se consideró $C_1=A^{-1}, C_2=B^{-1}$ y $C_3=C^{1/2}$.

In [5]:
def objective(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return ((4*b+c-4*a)**2)*(1/(A**2)+1/(B**2))
def constraint1(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return 14*a*(b-2*a)**4-1.5*(b**3)*(A**2)-1.5*(b+c-a)*a*b*(A**2)-1.5*a*(b**2)*(B**2)-0.75*(b+c-a)*(A**4)-0.125*b*(A**2)*(B**2)
def constraint2(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return 6*a*(b-2*a)**2-1.5*b*(A**2)-0.75*a*(B**2)
def constraint3(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return 4*(b-2*a)**3-0.25*(b+c-a)*(A**2)-3*(a**2)/((b-2*a)*C)-2*(b-2*a)*(B**2)
def constraint4(x):
    return x[1]-2*x[0]

Se da un punto inicial $x_0$ y se definen las restricciones del problema.

In [6]:
x0=[1, 8, 0.5, 3, 3]

#restricciones
con1={'type':'ineq', 'fun': constraint1}
con2={'type':'ineq', 'fun': constraint2}
con3={'type':'ineq', 'fun': constraint3}
con4={'type':'ineq', 'fun': constraint4}
cons=[con1, con2, con3, con4]

### Resolución

Fijamos $C_3$ suficientemente grande para resolver el problema. Debido a la libertad de las variables, resolvemos el problema $(P)$ a medida que iteramos sobre las cotas inferiores de $C_1$ y $C_2$. Por la naturaleza del problema original, podemos fijar cotas inferiores para $a, b$ y $c$.

In [7]:
C=10000000 # 10^7
lista=np.linspace(1, 80, 400).tolist()
val_p_min=100
sol_p_min=[10, 10, 10, 10, 10]

for j in lista:
    b1=(1, j)
    b=(0.01, None)
    # Cotas de las variables
    bnds=(b, b, b, b1, b1)
    # No definimos tipo de solver
    sol=minimize(objective, x0, bounds=bnds, constraints=cons)
    # Nos aseguramos de que, en efecto, se satisfagan las restricciones
    # Guardamos la mínima solución
    if sol.fun<val_p_min and constraint1(sol.x)>0 and constraint2(sol.x)>0 and constraint3(sol.x)>0 and constraint4(sol.x)>0:
        val_p_min=sol.fun
        sol_p_min=sol.x
        
print("---------------------------- Solución ----------------------------")
print("val(P)=", val_p_min)
print("Solución del problema: a=%s, b=%s, c=%s, C1=1/%s, C2=1/%s" %tuple(sol_p_min))
print("El valor de K1 es ", (4*sol_p_min[1]+sol_p_min[2]-4*sol_p_min[0])*(1/sol_p_min[3]))
print("El valor de K2 es ", (4*sol_p_min[1]+sol_p_min[2]-4*sol_p_min[0])*(1/sol_p_min[4]))

---------------------------- Solución ----------------------------
val(P)= 71.22234490163238
Solución del problema: a=10.392314734393258, b=90.68876636523513, c=0.01, C1=1/43.30119620959022, C2=1/79.80200501253132
El valor de K1 es  7.417712087414111
El valor de K2 es  4.024908979078033


In [13]:
print(objective(sol_p_min))
print(constraint1(sol_p_min))
print(constraint2(sol_p_min))
print(constraint3(sol_p_min))

71.22234490163238
0.0004653334617614746
1.4260876923799515e-09
438378.02573545603


### Comportamiento de $val(P)$

Al disminuir la cota inferior de $C_1$ y $C_2$ vemos que $val(P)$ tiene chance de decrecer, debido a que la función objetivo es decreciente respecto a dichos parámetros. En efecto, en la siguiente iteración, fijando como cota inferior $10^3$ podemos apreciar tal comportamiento, y a su vez, obtener la misma conclusión tanto para $K_1$ como para $K_2$.

In [5]:
lista=np.linspace(1, 1000).tolist()
for j in lista:
    b1=(1, j)
    b=(0.01, None)
    # Cotas de las variables
    bnds=(b, b, b, b1, b1)
    # No definimos tipo de solver
    sol=minimize(objective, x0, bounds=bnds, constraints=cons)
    # Nos aseguramos de que, en efecto, se satisfagan las restricciones
    if constraint1(sol.x)>0 and constraint2(sol.x)>0 and constraint3(sol.x)>0 and constraint4(sol.x)>0:
        print("---------------------------- Nueva iteración ----------------------------")
        print("val(P)=", sol.fun)
        print("El valor de K1 es ", (4*sol.x[1]+sol.x[2]-4*sol.x[0])*(1/sol.x[3]))
        print("El valor de K2 es ", (4*sol.x[1]+sol.x[2]-4*sol.x[0])*(1/sol.x[4]))

---------------------------- Nueva iteración ----------------------------
val(P)= 71.23647264814481
El valor de K1 es  7.418274734235644
El valor de K2 es  4.025626984153636
---------------------------- Nueva iteración ----------------------------
val(P)= 71.22705528403628
El valor de K1 es  7.417941486206812
El valor de K2 es  4.025071352317642
---------------------------- Nueva iteración ----------------------------
val(P)= 71.2238126268895
El valor de K1 es  7.417769746508723
El valor de K2 es  4.024985045272767
---------------------------- Nueva iteración ----------------------------
val(P)= 71.2211822938178
El valor de K1 es  7.4176650623190605
El valor de K2 es  4.024851216761801
---------------------------- Nueva iteración ----------------------------
val(P)= 71.22259753929497
El valor de K1 es  7.4177585492182265
El valor de K2 es  4.024854735862509
---------------------------- Nueva iteración ----------------------------
val(P)= 71.22067511014481
El valor de K1 es  7.417677195

## Conclusiones

Para $C_1, C_2\geq 1/80$, la elección aproximada de parámetros a tres decimales
$$a=13.426, b=70.3, c=0.01, C_1=1/33.191, C_2=1/59.704$$
nos entrega $$val(P_{aprox})\thickapprox 61.5039,$$
que es mayor a $val(P)$ y difiere en $2.6\cdot 10^{-3}$. A partir de ello, en nuestro problema principal se obtiene $\eta\geq 39.04$.

Se conjetura que $val(P)>61.49$ (en base a una cota inferior para $C_1, C_2$ de $10^{-3}$). Al disminuir la cota inferior de las variables $C_1$ y $C_2$, las mejoras obtenidas en la cota inferior del parámetro $\eta$ son del orden de $10^{-3}$, lo cual consideramos no significativo. Considerar además que el solver puede no dar buenos resultados con números de orden menor.

Asimismo, al iterar sobre distintos valores de cotas inferiores (sobre los parámetros del polinomio en particular) se ve que $K_1$ y $K_2$ decrecen, pero no de manera significativa, manteniéndose acotados inferiormente por $6.85$ y $3.81$ respectivamente.

Por último, mencionar que el problema es altamente sensible a la condición inicial $x_0$ asignada.